# Gensim Word2Vec Tutorial


<img src="https://images.freeimages.com/images/large-previews/2b9/letters-1-yes-1188348.jpg" alt="drawing" width="350"/>

# Motivation

As I started working at [Supportiv](http://www.supportiv.com), the support network for instant peer support, a few months ago, I began looking into Language Models and Word2Vec particularly. A python native, I naturally decided to focus on Gensim's implementation of Word2Vec, and went on to look for tutorials on the web. As all good data scientists, I directly applied and reproduced the code samples from multiple website. Confused and often disappointed by the results I got, I went deeper and deeper, from stackoverflow threads, to Gensim's Google Groups, onto the documentation of the library, to try and understand what went wrong in my approach. After weeks of hard labor, I finally managed to get decent results, but I was frustrated by these online tutorials which were, for the most part, misleading.

What troubled me the most in these online tutorials was their mismanagement of the model training: the code worked, and I got results which appeared to be decent at first, but the more I looked into them, the more disturbing they were. I wasted a lot of time figuring out what was wrong.

Another issue I had with these tutorials was the data preparation step: too often, the authors chose to load an existing preprocessed dataset, use a toy example or skip this part. However, I always thought that one of the most important parts of the creation of a Word2Vec model was then missing. During my experimentations, I noticed that lemmatizing the sentences or looking for phrases/bigrams in them had a big impact over the results and performance of my models. Though the influence of the preprocessing varies with each dataset and application, I thought I would include the data preparation steps in this tutorial and use the great spaCy library along with it.

I am not the only one annoyed by some of these [issues](https://groups.google.com/d/msg/gensim/jom4JFt7EV8/y5fjhupbAgAJ), so I decided to write my own tutorial.

I do not pledge that it is perfect, nor the best way to implement Word2Vec, simply that it is better than a good chunk of what is out there 😉

# Plan

1. [Briefing about Word2Vec](#Briefing-about-Word2Vec:)
    * [Purpose of the tutorial](#Purpose-of-the-tutorial:)
    * [Brief explanation](#Brief-explanation:)

2. [Getting Started](#Getting-Started)
    * [Setting up the environment](#Setting-up-the-environment:)
    * [The data](#The-data:)
3. [Preprocessing](#Preprocessing)
    * [Cleaning](#Cleaning)
    * [Bigrams](#Bigrams)
    * [Most frequent words](#Most-Frequent-Words)
    
4. [Training the Model](#Training-the-model)
    * [Gensim Word2Vec Implementation](#Gensim-Word2Vec-Implementation:)
    * [Why I seperate the training of the model in 3 steps](#Why-I-seperate-the-training-of-the-model-in-3-steps:)
    * [Training the model](#Training-the-model)
        * [The parameters](#The-parameters)
        * [Building the vocabulary table](#Building-the-Vocabulary-Table)
        * [Training of the model](#Training-of-the-model)
        * [Saving the model](#Saving-the-model:)
5. [Exploring the Model](#Exploring-the-model)
    * [Most similar to](#Most-similar-to:)
    * [Similarities](#Similarities:)
    * [Odd-one-out](#Odd-One-Out:)
    * [Analogy difference](#Analogy-difference:)
    * [t-SNE visualizations](#t-SNE-visualizations:)
        * [10 Most similar words vs. 8 Random words](#10-Most-similar-words-vs.-8-Random-words:)
        * [10 Most similar words vs. 10 Most dissimilar](#10-Most-similar-words-vs.-10-Most-dissimilar:)
        * [10 Most similar words vs. 11th to 20th Most similar words](#10-Most-similar-words-vs.-11th-to-20th-Most-similar-words:)
6. [Final Thoughts](#Final-Thoughts)
7. [Material for more in depths understanding](#Material-for-more-in-depths-understanding:)
8. [Acknowledgements](#Acknowledgements)
9. [References](#References:)
10. [End](#End)

# Briefing about Word2Vec:

<img src="http://mccormickml.com/assets/word2vec/skip_gram_net_arch.png" alt="drawing" width="550"/>

[[1]](#References:)


## Purpose of the tutorial:
As I said before, this tutorial focuses on the right use of the Word2Vec package from the Gensim libray; therefore, I am not going to explain the concepts and ideas behind Word2Vec here. I am simply going to give a very brief explanation, and provide you with links to good, in depth tutorials.

## Brief explanation:

Word2Vec was introduced in two [papers](#Material-for-more-in-depths-understanding:) between September and October 2013, by a team of researchers at Google. Along with the papers, the researchers published their implementation in C. The Python implementation was done soon after the 1st paper, by [Gensim](https://radimrehurek.com/gensim/index.html). 

The underlying assumption of Word2Vec is that two words sharing similar contexts also share a similar meaning and consequently a similar vector representation from the model. For instance: "dog", "puppy" and "pup" are often used in similar situations, with similar surrounding words like "good", "fluffy" or "cute", and according to Word2Vec they will therefore share a similar vector representation.<br>

From this assumption, Word2Vec can be used to find out the relations between words in a dataset, compute the similarity between them, or use the vector representation of those words as input for other applications such as text classification or clustering.

# Getting Started

## Setting up the environment:

`python==3.6.3`

Libraries used:
 * `xlrd==1.1.0`: https://pypi.org/project/xlrd/
 * `spaCy==2.0.12`: https://spacy.io/usage/
 * `gensim==3.4.0`: https://radimrehurek.com/gensim/install.html
 * `scikit-learn==0.19.1`: http://scikit-learn.org/stable/install.html
 * `seaborn==0.8`: https://seaborn.pydata.org/installing.html

In [2]:
!pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 KB 747.3 kB/s eta 0:00:0031m2.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 KB 952.7 kB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 KB 1.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.3/913.3 KB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 KB 2.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 5.2 MB/s eta 0:00:00m eta 0:00:0136m0:00:01


In [3]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

/home/saugat/Desktop/NLP/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/saugat/Desktop/NLP/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:651: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


<img src="https://upload.wikimedia.org/wikipedia/commons/4/44/Logo_The_Simpsons.svg" alt="drawing" width="400"/>

## The data:
I chose to play with the script from the Simpsons, both because I love the Simpsons and because with more than 150k lines of dialogues, the dataset was substantial!

This dataset contains the characters, locations, episode details, and script lines for approximately 600 Simpsons episodes, dating back to 1989. It can be found here: https://www.kaggle.com/ambarish/fun-in-text-mining-with-simpsons/data (~25MB)

# Preprocessing

We keep only two columns:
* `raw_character_text`: the character who speaks (can be useful when monitoring the preprocessing steps)
* `spoken_words`: the raw text from the line of dialogue

We do not keep `normalized_text` because we want to do our own preprocessing.

You can find the resulting file here: https://www.kaggle.com/pierremegret/dialogue-lines-of-the-simpsons

In [4]:
df = pd.read_csv('./simpsons_dataset.csv')
df.shape

(158314, 2)

In [5]:
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


The missing values comes from the part of the script where something happens, but with no dialogue. For instance "(Springfield Elementary School: EXT. ELEMENTARY - SCHOOL PLAYGROUND - AFTERNOON)"

In [6]:
df.isnull().sum()

raw_character_text    17814
spoken_words          26459
dtype: int64

Removing the missing values:

In [7]:
df = df.dropna().reset_index(drop=True)
df.isnull().sum()

raw_character_text    0
spoken_words          0
dtype: int64

## Cleaning:
We are lemmatizing and removing the stopwords and non-alphabetic characters for each line of dialogue.

In [13]:
#need to download the spacy model
# !python3 -m spacy download en_core_web_sm

In [15]:
# nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed
nlp = spacy.load("en_core_web_sm" , disable=['ner' , 'parser'])
def cleaning(doc):
    # Lemmatizes and removes stopwords , 
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

Removes non-alphabetic characters:

In [16]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['spoken_words'])

Taking advantage of spaCy .pipe() attribute to speed-up the cleaning process:

In [18]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 1.76 mins


Put the results in a DataFrame to remove missing values and duplicates:

In [19]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape
df_clean.head()

,clean
0,actually little disease magazine news show nat...
2,know sure like talk touch lesson plan teach
3,life worth live
4,poll open end recess case decide thought final...
7,victory party slide


## Bigrams:
We are using Gensim Phrases package to automatically detect common phrases (bigrams) from a list of sentences.
https://radimrehurek.com/gensim/models/phrases.html

The main reason we do this is to catch words like "mr_burns" or "bart_simpson" !

In [20]:
from gensim.models.phrases import Phrases, Phraser

As `Phrases()` takes a list of list of words as input:

In [21]:
sent = [row.split() for row in df_clean['clean']]

Creates the relevant phrases from the list of sentences:

In [22]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

INFO - 20:17:55: collecting all words and their counts
INFO - 20:17:55: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 20:17:55: PROGRESS: at sentence #10000, processed 63557 words and 52723 word types
INFO - 20:17:55: PROGRESS: at sentence #20000, processed 130936 words and 99612 word types
INFO - 20:17:55: PROGRESS: at sentence #30000, processed 192961 words and 138181 word types
INFO - 20:17:55: PROGRESS: at sentence #40000, processed 249832 words and 172156 word types
INFO - 20:17:55: PROGRESS: at sentence #50000, processed 311269 words and 207943 word types
INFO - 20:17:55: PROGRESS: at sentence #60000, processed 373578 words and 242950 word types
INFO - 20:17:55: PROGRESS: at sentence #70000, processed 436424 words and 277852 word types
INFO - 20:17:56: PROGRESS: at sentence #80000, processed 497887 words and 310927 word types
INFO - 20:17:56: collected 329641 token types (unigram + bigrams) from a corpus of 537095 words and 85955 sentences
INFO - 20:17:56: m

The goal of Phraser() is to cut down memory consumption of Phrases(), by discarding model state not strictly needed for the bigram detection task:

In [23]:
bigram = Phraser(phrases)

INFO - 20:18:11: exporting phrases from Phrases<329641 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 20:18:12: FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<128 phrases, min_count=30, threshold=10.0> from Phrases<329641 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000> in 0.56s', 'datetime': '2023-06-08T20:18:12.066169', 'gensim': '4.3.1', 'python': '3.10.6 (main, Mar 10 2023, 10:55:28) [GCC 11.3.0]', 'platform': 'Linux-5.19.0-43-generic-x86_64-with-glibc2.35', 'event': 'created'}


Transform the corpus based on the bigrams detected:

In [24]:
sentences = bigram[sent]

## Most Frequent Words:
Mainly a sanity check of the effectiveness of the lemmatization, removal of stopwords, and addition of bigrams.

In [25]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

29694

In [28]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['oh', 'like', 'know', 'get', 'hey', 'think', 'come', 'right', 'look', 'want']

# Training the model
## Gensim Word2Vec Implementation:
We use Gensim implementation of word2vec: https://radimrehurek.com/gensim/models/word2vec.html

In [29]:
import multiprocessing

from gensim.models import Word2Vec

## Why I seperate the training of the model in 3 steps:
I prefer to separate the training in 3 distinctive steps for clarity and monitoring.
1. `Word2Vec()`: 
>In this first step, I set up the parameters of the model one-by-one. <br>I do not supply the parameter `sentences`, and therefore leave the model uninitialized, purposefully.
2. `.build_vocab()`: 
>Here it builds the vocabulary from a sequence of sentences and thus initialized the model. <br>With the loggings, I can follow the progress and even more important, the effect of `min_count` and `sample` on the word corpus. I noticed that these two parameters, and in particular `sample`, have a great influence over the performance of a model. Displaying both allows for a more accurate and an easier management of their influence.
3. `.train()`:
>Finally, trains the model.<br>
The loggings here are mainly useful for monitoring, making sure that no threads are executed instantaneously.

In [31]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

8

## The parameters:

* `min_count` <font color='purple'>=</font> <font color='green'>int</font> - Ignores all words with total absolute frequency lower than this - (2, 100)


* `window` <font color='purple'>=</font> <font color='green'>int</font> - The maximum distance between the current and predicted word within a sentence. E.g. `window` words on the left and `window` words on the left of our target - (2, 10)


* `size` <font color='purple'>=</font> <font color='green'>int</font> - Dimensionality of the feature vectors. - (50, 300)


* `sample` <font color='purple'>=</font> <font color='green'>float</font> - The threshold for configuring which higher-frequency words are randomly downsampled. Highly influencial.  - (0, 1e-5)


* `alpha` <font color='purple'>=</font> <font color='green'>float</font> - The initial learning rate - (0.01, 0.05)


* `min_alpha` <font color='purple'>=</font> <font color='green'>float</font> - Learning rate will linearly drop to `min_alpha` as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00


* `negative` <font color='purple'>=</font> <font color='green'>int</font> - If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)


* `workers` <font color='purple'>=</font> <font color='green'>int</font> - Use these many worker threads to train the model (=faster training with multicore machines)

In [33]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

INFO - 20:21:48: Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=100, alpha=0.03>', 'datetime': '2023-06-08T20:21:48.388094', 'gensim': '4.3.1', 'python': '3.10.6 (main, Mar 10 2023, 10:55:28) [GCC 11.3.0]', 'platform': 'Linux-5.19.0-43-generic-x86_64-with-glibc2.35', 'event': 'created'}


## Building the Vocabulary Table:
Word2Vec requires us to build the vocabulary table (simply digesting all the words and filtering out the unique words, and doing some basic counts on them):

In [34]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 20:23:02: collecting all words and their counts
INFO - 20:23:02: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 20:23:02: PROGRESS: at sentence #10000, processed 61697 words, keeping 9518 word types
INFO - 20:23:02: PROGRESS: at sentence #20000, processed 127312 words, keeping 14384 word types
INFO - 20:23:02: PROGRESS: at sentence #30000, processed 187772 words, keeping 17442 word types
INFO - 20:23:02: PROGRESS: at sentence #40000, processed 243265 words, keeping 20121 word types
INFO - 20:23:02: PROGRESS: at sentence #50000, processed 303120 words, keeping 22551 word types
INFO - 20:23:02: PROGRESS: at sentence #60000, processed 363858 words, keeping 24820 word types
INFO - 20:23:02: PROGRESS: at sentence #70000, processed 425311 words, keeping 26987 word types
INFO - 20:23:02: PROGRESS: at sentence #80000, processed 485433 words, keeping 28822 word types
INFO - 20:23:02: collected 29694 word types from a corpus of 523538 raw words and 85955 sentence

Time to build vocab: 0.01 mins


## Training of the model:
_Parameters of the training:_
* `total_examples` <font color='purple'>=</font> <font color='green'>int</font> - Count of sentences;
* `epochs` <font color='purple'>=</font> <font color='green'>int</font> - Number of iterations (epochs) over the corpus - [10, 20, 30]

In [35]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 20:23:50: Word2Vec lifecycle event {'msg': 'training model with 7 workers on 3325 vocabulary and 100 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2 shrink_windows=True', 'datetime': '2023-06-08T20:23:50.299397', 'gensim': '4.3.1', 'python': '3.10.6 (main, Mar 10 2023, 10:55:28) [GCC 11.3.0]', 'platform': 'Linux-5.19.0-43-generic-x86_64-with-glibc2.35', 'event': 'train'}
INFO - 20:23:51: EPOCH 0: training on 523538 raw words (199488 effective words) took 0.7s, 270243 effective words/s
INFO - 20:23:52: EPOCH 1 - PROGRESS: at 91.99% examples, 180713 words/s, in_qsize 0, out_qsize 0
INFO - 20:23:52: EPOCH 1: training on 523538 raw words (199423 effective words) took 1.1s, 187434 effective words/s
INFO - 20:23:52: EPOCH 2: training on 523538 raw words (199123 effective words) took 0.7s, 275641 effective words/s
INFO - 20:23:53: EPOCH 3: training on 523538 raw words (199356 effective words) took 0.9s, 214100 effective words/s
INFO - 20:23:54: EPOCH 4: training on 523538 r

Time to train the model: 0.48 mins


As we do not plan to train the model any further, we are calling init_sims(), which will make the model much more memory-efficient:

In [36]:
w2v_model.init_sims(replace=True)

/tmp/ipykernel_195837/514372312.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)
WARNING - 20:24:45: destructive init_sims(replace=True) deprecated & no longer required for space-efficiency


# Exploring the model
## Most similar to:

Here, we will ask our model to find the word most similar to some of the most iconic characters of the Simpsons!

![Homer_Simpson](https://vignette.wikia.nocookie.net/simpsons/images/0/02/Homer_Simpson_2006.png/revision/latest?cb=20091207194310)

Let's see what we get for the show's main character:

In [37]:
w2v_model.wv.most_similar(positive=["homer"])

[('depressed', 0.7135162353515625),
 ('gee', 0.6696743965148926),
 ('creepy', 0.6566376686096191),
 ('marge', 0.6520886421203613),
 ('terrific', 0.6516083478927612),
 ('sweetheart', 0.643757700920105),
 ('hammock', 0.642941951751709),
 ('moe', 0.6417779922485352),
 ('shoulda', 0.6373036503791809),
 ('compliment', 0.6356023550033569)]

_A small precision here:_<br>
The dataset is the Simpsons' lines of dialogue; therefore, when we look at the most similar words from "homer" we do **not** necessary get his family members, personality traits, or even his most quotable words. No, we get what other characters (as Homer does not often refers to himself at the 3rd person) said along with "homer", such as how he feels or looks ("depressed"), where he is ("hammock"), or with whom ("marge").

Let's see what the bigram "homer_simpson" gives us by comparison:

In [38]:
w2v_model.wv.most_similar(positive=["homer_simpson"])

[('easily', 0.6567609906196594),
 ('select', 0.6506896018981934),
 ('candidate', 0.6455971002578735),
 ('poll', 0.6428175568580627),
 ('montgomery_burn', 0.6419366598129272),
 ('debate', 0.6270120143890381),
 ('committee', 0.6242663860321045),
 ('governor', 0.6197243928909302),
 ('winner', 0.6191943883895874),
 ('rainy', 0.6168389916419983)]

![Marge](https://vignette.wikia.nocookie.net/simpsons/images/0/0b/Marge_Simpson.png/revision/latest?cb=20180626055729)

What about Marge now?

In [39]:
w2v_model.wv.most_similar(positive=["marge"])

[('snuggle', 0.6661581993103027),
 ('homer', 0.6520886421203613),
 ('ned', 0.6509064435958862),
 ('patty_selma', 0.6418110728263855),
 ('decent', 0.6397137641906738),
 ('rapture', 0.6393093466758728),
 ('marry', 0.6378631591796875),
 ('badly', 0.6355270743370056),
 ('awww', 0.6318559646606445),
 ('maybe', 0.631703794002533)]

![bart](https://vignette.wikia.nocookie.net/simpsons/images/6/65/Bart_Simpson.png/revision/latest?cb=20180319061933)

Let's check Bart now:

In [41]:
w2v_model.wv.most_similar(positive=["bart"])

[('lisa', 0.7992392778396606),
 ('mom_dad', 0.691601037979126),
 ('hearing', 0.6710435152053833),
 ('mom', 0.6702009439468384),
 ('strangle', 0.668476939201355),
 ('exploit', 0.6660684943199158),
 ('convince', 0.6627126336097717),
 ('milhouse', 0.6582393646240234),
 ('pay_attention', 0.6505537033081055),
 ('substitute', 0.6499829292297363)]

Looks like it is making sense!
![Groundskeeper_Willie](https://vignette.wikia.nocookie.net/simpsons/images/9/9d/Groundskeeper_Willie.png/revision/latest?cb=20130424154035)

Willie the groundskeeper for the last one:

## Similarities:
Here, we will see how similar are two words to each other :

In [42]:
w2v_model.wv.similarity("moe_'s", 'tavern')

KeyError: "Key 'moe_'s' not present"

Who could forget Moe's tavern? Not Barney.

![](https://vignette.wikia.nocookie.net/simpsons/images/6/6c/MaggieSimpson.PNG/revision/latest?cb=20180314210204) 

In [43]:
w2v_model.wv.similarity('maggie', 'baby')

0.5947409

Maggie is indeed the most renown baby in the Simpsons!

In [44]:
w2v_model.wv.similarity('bart', 'nelson')

0.49807742

Bart and Nelson, though friends, are not that close, makes sense!

## Odd-One-Out:

Here, we ask our model to give us the word that does not belong to the list!

Between Jimbo, Milhouse, and Kearney, who is the one who is not a bully?

In [45]:
w2v_model.wv.doesnt_match(['jimbo', 'milhouse', 'kearney'])

WARNING - 20:30:43: vectors for words {'kearney'} are not present in the model, ignoring these words


'jimbo'

Milhouse of course!
![](https://vignette.wikia.nocookie.net/simpsons/images/9/91/Milhouse_Van_Houten_2.png/revision/latest?cb=20180429212659)

What if we compared the friendship between Nelson, Bart, and Milhouse?

In [46]:
w2v_model.wv.doesnt_match(["nelson", "bart", "milhouse"])

'nelson'

Seems like Nelson is the odd one here!
![](https://vignette.wikia.nocookie.net/simpsons/images/4/40/Picture0003.jpg/revision/latest?cb=20110623042517)

Last but not least, how is the relationship between Homer and his two sister-in-laws?

In [47]:
w2v_model.wv.doesnt_match(['homer', 'patty', 'selma'])

'homer'

Damn, they really do not like you Homer!

## Analogy difference:
Which word is to woman as homer is to marge?

In [48]:
w2v_model.wv.most_similar(positive=["woman", "homer"], negative=["marge"], topn=3)

[('reason', 0.5643137693405151),
 ('certain', 0.5641007423400879),
 ('see', 0.5545346736907959)]

"man" comes at the first position, that looks about right!

Which word is to woman as bart is to man?

In [ ]:
w2v_model.wv.most_similar(positive=["woman", "bart"], negative=["man"], topn=3)

Lisa is Bart's sister, her male counterpart!
<img src="https://vignette.wikia.nocookie.net/simpsons/images/5/57/Lisa_Simpson2.png/revision/latest?cb=20180319000458" alt="drawing" width="100"/>

### t-SNE visualizations:
t-SNE is a non-linear dimensionality reduction algorithm that attempts to represent high-dimensional data and the underlying relationships between vectors in a lower-dimensional space.<br>
Here is a good tutorial on it: https://medium.com/@luckylwk/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b

In [50]:
!pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 KB 1.3 MB/s eta 0:00:000:00:01m eta 0:00:01


In [51]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

Our goal in this section is to plot our 300 dimensions vectors into 2 dimensional graphs, and see if we can spot interesting patterns.<br>
For that we are going to use t-SNE implementation from scikit-learn.

To make the visualizations more relevant, we will look at the relationships between a query word (in <font color='red'>**red**</font>), its most similar words in the model (in <font color="blue">**blue**</font>), and other words from the vocabulary (in <font color='green'>**green**</font>).

In [52]:
def tsnescatterplot(model, word, list_names):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = [word]
    color_list  = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, model.wv.__getitem__([word]), axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar([word])
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)
        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=50).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('t-SNE visualization for {}'.format(word.title()))
    

Code inspired by: [[2]](#References:)

## 10 Most similar words vs. 8 Random words:
Let's compare where the vector representation of Homer, his 10 most similar words from the model, as well as 8 random ones, lies in a 2D graph:

In [53]:
tsnescatterplot(w2v_model, 'homer', ['dog', 'bird', 'ah', 'maude', 'bob', 'mel', 'apu', 'duff'])

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 300 and the array at index 1 has size 100

Interestingly, the 10 most similar words to Homer ends up around him, so does Apu and (sideshow) Bob, two recurrent characters.

## 10 Most similar words vs. 10 Most dissimilar

This time, let's compare where the vector representation of Maggie and her 10 most similar words from the model lies compare to the vector representation of the 10 most dissimilar words to Maggie:

In [54]:
tsnescatterplot(w2v_model, 'maggie', [i[0] for i in w2v_model.wv.most_similar(negative=["maggie"])])

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 300 and the array at index 1 has size 100

Neat! Maggie and her most similar words form a distinctive cluster from the most dissimilar words, it is a really encouraging plot!

## 10 Most similar words vs. 11th to 20th Most similar words:

Finally, we are going to plot the most similar words to Mr. Burns ranked 1st to 10th versus the ones ranked 11th to 20th:

(PS: Mr. Burns became mr_burn after the preprocessing)

In [55]:
tsnescatterplot(w2v_model, "mr_burn", [t[0] for t in w2v_model.wv.most_similar(positive=["mr_burn"], topn=20)][10:])

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 300 and the array at index 1 has size 100

As we can see, and that is very nice, all the 20 words are forming one cluster around Mr. Burns.

# Final Thoughts

I hope you found this tutorial useful and had as much fun reading it as I had writing it. Please do not hesitate to leave any comments, questions or suggestions you might have. See you around!

Also, please check [Supportiv](http://www.supportiv.com) around! (Simpson-ized logo)

<img src="https://fontmeme.com/permalink/180904/cc3d27a8aaa88189e764ee9d02331d0d.png" alt="drawing" width="500"/>


# Materials for more in depths understanding:
* Word Embeddings introduction: https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/
* Word2Vec introduction: https://skymind.ai/wiki/word2vec
* Another Word2Vec introduction: http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
* A great Gensim implentation tutorial: http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W467ScBjM2x
* Original articles from Mikolov et al.: https://arxiv.org/abs/1301.3781 and https://arxiv.org/abs/1310.4546


# Acknowledgements

* [Pouria Mojabi](https://www.linkedin.com/in/pouria-mojabi-1873615/), co-fouder of Supportiv Inc.

# References:
* [1]. Neural Net picture: McCormick, C. (2016, April 19). Word2Vec Tutorial - The Skip-Gram Model. Retrieved from http://www.mccormickml.com
* [2]. Aneesha Bakharia Medium article: https://medium.com/@aneesha/using-tsne-to-plot-a-subset-of-similar-words-from-word2vec-bb8eeaea6229

# End